In [26]:
cd /home/jovyan/shares/SR004.nfs2/skripkin/speech2latex/audio-flamingo

/home/jovyan/shares/SR004.nfs2/skripkin/speech2latex/audio-flamingo


In [2]:
pwd

'/home/jovyan/shares/SR004.nfs2/skripkin/speech2latex/audio-flamingo'

In [3]:
from tqdm.auto import tqdm

/home/jovyan/.mlspace/envs/audio_flamingo_3_skripkin/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import argparse
import importlib.util
import json
import os

from pydantic import BaseModel
from termcolor import colored

import llava
from llava import conversation as clib
from llava.media import Image, Video, Sound
from llava.model.configuration_llava import JsonSchemaResponseFormat, ResponseFormat
from peft import PeftModel
import torch

[2025-07-30 23:36:22,642] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/jovyan/.mlspace/envs/audio_flamingo_3_skripkin/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/jovyan/.mlspace/envs/audio_flamingo_3_skripkin/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


In [5]:
def get_schema_from_python_path(path: str) -> str:
    schema_path = os.path.abspath(path)
    spec = importlib.util.spec_from_file_location("schema_module", schema_path)
    schema_module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(schema_module)

    # Get the Main class from the loaded module
    Main = schema_module.Main
    assert issubclass(
        Main, BaseModel
    ), f"The provided python file {path} does not contain a class Main that describes a JSON schema"
    return Main.schema_json()

In [6]:
data_test = json.load(open('./data/Speech2Latex_equations_test_audio_flamingo_3.json'))

In [7]:
# model = llava.load("/home/jovyan/shares/SR004.nfs2/skripkin/speech2latex/audio-flamingo/runs/train/finetune_on_speech2latex_equations/")
model = llava.load("/home/jovyan/shares/SR004.nfs2/skripkin/speech2latex/audio-flamingo/runs/train/finetune_on_speech2latex_equations_run_2/checkpoint-600/")

['/home/jovyan/shares/SR004.nfs2/skripkin/speech2latex/audio-flamingo/runs/train/finetune_on_speech2latex_equations_run_2/checkpoint-600/llm', 'Efficient-Large-Model/paligemma-siglip-so400m-patch14-448', 'openai/whisper-large-v2', '/home/jovyan/shares/SR004.nfs2/skripkin/speech2latex/audio-flamingo/runs/train/finetune_on_speech2latex_equations_run_2/checkpoint-600/sound_tower', 'mlp_downsample', 'mlp', '/home/jovyan/shares/SR004.nfs2/skripkin/speech2latex/audio-flamingo/runs/train/finetune_on_speech2latex_equations_run_2/checkpoint-600/sound_mm_projector']


Loading checkpoint shards: 100%|██████████| 4/4 [01:33<00:00, 23.27s/it]


/home/jovyan/shares/SR004.nfs2/skripkin/speech2latex/audio-flamingo/runs/train/finetune_on_speech2latex_equations_run_2/checkpoint-600/


In [8]:
text = ''
clib.default_conversation = clib.conv_templates["auto"].copy()

In [ ]:
# python llava/cli/infer_audio.py 
# --model-base /home/jovyan/shares/SR004.nfs2/skripkin/speech2latex/audio-flamingo/runs/train/finetune_on_speech2latex_equations/ 
# --conv-mode auto 
# --text "" 
# --media /home/jovyan/shares/SR004.nfs2/skripkin/speech2latex/audio-flamingo/data/equations_test/7.wav

In [ ]:
# parser = argparse.ArgumentParser()
# parser.add_argument("--model-base", "-m", type=str, required=True)
# parser.add_argument("--conv-mode", "-c", type=str, default="auto")
# parser.add_argument("--text", type=str)
# parser.add_argument("--media", type=str, nargs="+")
# parser.add_argument("--json-mode", action="store_true")
# parser.add_argument("--think-mode", action="store_true")
# parser.add_argument("--json-schema", type=str, default=None)
# args = parser.parse_args()

In [9]:
# if not args.json_mode:
#     response_format = None
# elif args.json_schema is None:
#     response_format = ResponseFormat(type="json_object")
# else:
#     schema_str = get_schema_from_python_path(args.json_schema)
#     print(schema_str)
#     response_format = ResponseFormat(type="json_schema", json_schema=JsonSchemaResponseFormat(schema=schema_str))

In [10]:
n = 2
sample = data_test[n]
sample

{'id': 'ID2',
 'sound': '/home/jovyan/shares/SR004.nfs2/skripkin/speech2latex/audio-flamingo/data/equations_test/2.wav',
 'duration': 9.888666666666667,
 'conversations': [{'from': 'human', 'value': '<sound>'},
  {'from': 'gpt', 'value': '(y_{1},y_{2},y_{3})=(\\phi,\\psi,1)'}]}

In [11]:
media = Sound(sample['sound'])
prompt = [media, text]

In [12]:
response = model.generate_content(prompt, response_format=None)

/home/jovyan/shares/SR004.nfs2/skripkin/speech2latex/audio-flamingo/llava/mm_utils.py:592: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  masks = torch.tensor(masks[0])


In [13]:
response

'(y_{1},y_{2},y_{3})=(\\xi,c,s_{1})'

In [35]:
sample['conversations'][1]['value']

'(y_{1},y_{2},y_{3})=(\\phi,\\psi,1)'

In [ ]:
latex_true, latex_pred = [], []
outputs = {
    'latex_true': [],
    'latex_pred': []
}
with torch.no_grad():
    for sample in tqdm(data_test):
        media = Sound(sample['sound'])
        prompt = [media, text]
        response = model.generate_content(prompt, response_format=None)
        outputs['latex_true'].append(sample['conversations'][1]['value'])
        outputs['latex_pred'].append(response)

  0%|          | 9/2880 [01:27<7:39:26,  9.60s/it]

In [41]:
outputs['latex_pred']

['y^{2}=x^{3}-8x',
 '\\text{eigenvectorsof}A\\text{aresrootsof}\\text{ch}_{A}(\\lambda)',
 '(y_{1},y_{2},y_{3})=(\\phi,c,s_{1})',
 '\\frac{d^{2}y}{dx^{2}}=-y',
 '\\hat{\\mu}-\\hat{\\nu}',
 '\\beta',
 'w:=u-u^{\\prime}',
 '\\widetilde{\\mu}:=\\{\\mu_{1}+\\mu_{\\widetilde{1}},\\mu_{2},\\dots,\\mu_{d}\\}',
 'W^{-}',
 'Y_{i,j}=y_{i,j}(z_{i,j},h_{j})',
 'y_{i,i}+y_{j,j}=y_{h,h}+y_{g,g}',
 'e^{x}=\\sum_{n=0}^{\\infty}\\frac{x^{n}}{n!}',
 'Y_{i},X_{i}',
 '\\frac{\\partial\\\\L}{\\partial\\phi}-\\frac{d}{dx^{\\mu}}\\frac{\\partial\\\\L}{\\partial(d\\phi/dx^{\\mu})}=0',
 'w_{d_{1}},\\ldots,w_{d_{q}}',
 '\\frac{\\partial\\\\L}{\\partial\\phi}-\\frac{d}{dx^{\\mu}}\\frac{\\partial\\\\L}{\\partial(d\\phi/dx^{\\mu})}=0',
 'Y_{j}=\\frac{Z_{j}}{N}',
 'N_{G}(\\mathcal\\\\S_{1})\\cup\\\\N_{G}(\\mathcal\\\\S_{2})\\setminus{c_{1},c_{2}}',
 'w_{2}',
 'N(1_{Q}^{T},4_{I_{n}})',
 'Y_{n}=X_{n}-X_{n}(\\det\\\\S_{n})',
 '{\\alpha_{n}}_{n\\geqslant\\\\1}',
 '\\mathcal\\\\E(T^{\\prime})\\subset\\mathcal\\\\E(T)',


In [42]:
sample

{'id': 'ID2879',
 'sound': '/home/jovyan/shares/SR004.nfs2/skripkin/speech2latex/audio-flamingo/data/equations_test/2879.wav',
 'duration': 2.22,
 'conversations': [{'from': 'human', 'value': '<sound>'},
  {'from': 'gpt', 'value': '\\vec{w}'}]}

In [43]:
outputs['latex_true'] = [i['conversations'][1]['value'] for i in data_test]

In [17]:
outputs['latex_true']

['y^{2}=x^{3}-8x',
 '\\text{det}(A-\\lambda\\\\I)=0',
 '(y_{1},y_{2},y_{3})=(\\phi,\\psi,1)',
 '\\frac{d^{2}y}{dx^{2}}=-y',
 '\\hat{\\mu}-\\hat{\\nu}',
 '\\beta',
 'w:=u-u^{\\prime}',
 '\\tilde{\\mu}:={\\mu_{1}+\\tilde{\\mu}_{1},\\mu_{2},\\dots,\\mu_{d}}',
 'W_{-}',
 'Y_{i,g}=Y_{i,g}(Z_{i,g},H_{g})',
 'y_{I,I}+y_{J,J}=y_{M,M}+y_{H,H}',
 'e^{x}=\\sum_{n=0}^{\\infty}\\frac{x^{n}}{n!}',
 '(Y_{i},X_{i})',
 '\\frac{\\partial\\\\L}{\\partial\\phi}-\\frac{d}{dx^{\\mu}}\\frac{\\partial\\\\L}{\\partial(d\\phi/d\\\\x^{\\mu})}=0',
 'W_{D}^{1},\\cdots,W_{D}^{q}',
 '\\frac{\\partial\\\\L}{\\partial\\phi}-\\frac{d}{dx^{\\mu}}\\frac{\\partial\\\\L}{\\partial(d\\phi/d\\\\x^{\\mu})}=0',
 'y_{j}=z_{j}/N',
 'N_{G}(s_{1})\\cup\\\\N_{G}(s_{2})\\setminus{s_{1},s_{2}}',
 'W_{2}',
 '\\mathcal\\\\N\\left({1_{Q}^{\\top},4{I_{n}}}\\right)',
 'Y_{n}=X_{n}-X_{n}^{\\det}',
 '{\\alpha_{n}}_{n\\geq\\\\1}',
 '\\mathcal\\\\E(T^{\\prime})\\subset\\mathcal\\\\E(T)',
 '\\chi_{e}^{\\delta(X)}=1',
 '\\frac{7}{16}',
 'f(x)=\

In [18]:
outputs['latex_pred']

['y^{2}=x^{3}-8x',
 '\\text{Aigenvaluesare}\\text{coefficientsof}\\text{polynomial}',
 '(y_{1},y_{2},y_{3})=(\\xi,c,s_{1})',
 '\\frac{d^{2}y}{dx^{2}}=-y',
 '\\hat{\\mu}-\\hat{\\nu}',
 '\\beta',
 'w:=u-u^{\\prime}',
 '\\mu_{\\tilde{d}}:=\\{\\mu_{1}+\\mu_{\\tilde{1}},\\mu_{2},\\dots,\\mu_{d}\\}',
 'W^{-}',
 'y_{i,j}=y_{i,j}(z_{i,j},h_{j})',
 'y_{i,i}+y_{j,j}=y_{h,h}+y_{g,g}',
 'e^{x}=\\sum_{n=0}^{\\infty}\\frac{x^{n}}{n!}',
 '(y_{i},x_{i})',
 '\\frac{\\partial\\mathcal\\\\L}{\\partial\\phi}-\\frac{d}{dx^{\\mu}}\\left(\\frac{\\partial\\mathcal\\\\L}{\\partial(d\\phi)}\\frac{d\\phi}{dx^{\\mu}}\\right)=0',
 'W_{d1},\\ldots,W_{dq}',
 '\\frac{\\partial\\mathcal\\\\L}{\\partial\\phi}-\\frac{d}{dx^{\\mu}}\\left(\\frac{\\partial\\mathcal\\\\L}{\\partial(d\\phi)}\\frac{d}{dx^{\\mu}}\\right)=0',
 'Y_{j}=\\mathcal\\\\Z_{j}/N',
 'Nl_{g}(\\{s_{1}\\cup\\\\Nl_{g}(\\{s_{2}\\})\\})\\setminus{c_{1},c_{2}}',
 'w_{2}',
 'n(1_{q}^{T},4_{i_{n}})',
 'y_{n}=x_{n}-x\\det_{n}',
 '{\\alpha_{n}}_{n\\geq\\\\1}',
 '\

In [19]:
from IPython.display import Latex

In [20]:
n = 10
Latex(f"${outputs['latex_pred'][n]}$")

<IPython.core.display.Latex object>

In [22]:
Latex(f"${outputs['latex_true'][n]}$")

<IPython.core.display.Latex object>

In [23]:
import pandas as pd

In [24]:
df_test = pd.DataFrame(outputs)
df_test

,latex_true,latex_pred
0,y^{2}=x^{3}-8x,y^{2}=x^{3}-8x
1,\text{det}(A-\lambda\\I)=0,\text{Aigenvaluesare}\text{coefficientsof}\tex...
2,"(y_{1},y_{2},y_{3})=(\phi,\psi,1)","(y_{1},y_{2},y_{3})=(\xi,c,s_{1})"
3,\frac{d^{2}y}{dx^{2}}=-y,\frac{d^{2}y}{dx^{2}}=-y
4,\hat{\mu}-\hat{\nu},\hat{\mu}-\hat{\nu}
...,...,...
2875,"\vec{k}=(1,1,0,0)","\vec{k}=(1,1,0,0)"
2876,"\vec{k}=(1,1,0,0)","\vec{k}=(1,1,0,0)"
2877,C_{0},t_{0}
2878,\vec{v}_{H},\vec{v}_{\text{up}}


In [25]:
df_test.to_csv('pred_true_audio_flamingo_3_equations_test_chkp_600.csv', index = False)

In [46]:
pwd

'/home/jovyan/shares/SR004.nfs2/skripkin/speech2latex/audio-flamingo'

In [49]:
json.dump(outputs, open(f'pred_true_audio_flamingo_3_equations_test.json', 'w+'))

In [84]:
import evaluate

# Загрузка метрик
cer_metric = evaluate.load("cer")
wer_metric = evaluate.load("wer")

# Пример данных
references = outputs['latex_true']  # эталонный текст
predictions = outputs['latex_pred']  # предсказанный текст

# Расчет CER (Character Error Rate)
cer_score = cer_metric.compute(predictions=predictions, references=references)
print(f"CER: {cer_score:.4f}")

# Расчет WER (Word Error Rate)
wer_score = wer_metric.compute(predictions=predictions, references=references)
print(f"WER: {wer_score:.4f}")

CER: 0.4178
WER: 0.6019
